# Systematic Macro Credit — End-to-End Workflow

This notebook demonstrates the complete workflow for evaluating a single CDX signal:

1. **Data Generation**: Create synthetic CDX and ETF data
2. **Data Loading**: Load and validate market data
3. **Signal Computation**: Calculate cdx_etf_basis signal
4. **Backtesting**: Run strategy backtest with position tracking
5. **Visualization**: Analyze results with interactive plots

All components use deterministic random seeds for reproducibility.

## 1. Setup and Imports

In [ ]:
import logging
from pathlib import Path

# Configure logging for notebook (not in library code)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Import project modules
from macrocredit.data.sample_data import (
    generate_cdx_sample,
    generate_etf_sample,
    generate_vix_sample,
)
from macrocredit.models.signals import compute_cdx_etf_basis
from macrocredit.models.config import SignalConfig
from macrocredit.backtest.config import BacktestConfig
from macrocredit.backtest.engine import run_backtest
from macrocredit.backtest.metrics import compute_performance_metrics
from macrocredit.visualization.plots import (
    plot_signal,
    plot_equity_curve,
    plot_drawdown,
)

print("Imports successful")

Imports successful


## 2. Generate Sample Data

Create synthetic market data with realistic dynamics:
- **CDX IG 5Y**: Investment grade credit spreads
- **HYG ETF**: High yield credit ETF

In [ ]:
# Configuration
START_DATE = "2023-01-01"
PERIODS = 252  # One year of trading days
SEED = 42

# Generate CDX data
cdx_data = generate_cdx_sample(
    start_date=START_DATE,
    periods=PERIODS,
    index_name="CDX_IG",
    tenor="5Y",
    base_spread=70.0,
    volatility=3.0,
    seed=SEED,
)

# Generate ETF data
etf_data = generate_etf_sample(
    start_date=START_DATE,
    periods=PERIODS,
    ticker="HYG",
    base_price=75.0,
    volatility=0.6,
    seed=SEED + 1,
)

# Generate VIX data
vix_data = generate_vix_sample(
    start_date=START_DATE,
    periods=PERIODS,
    base_vix=15.0,
    volatility=2.0,
    seed=SEED + 2,
)

print(f"Generated {len(cdx_data)} days of CDX data")
print(f"Generated {len(etf_data)} days of ETF data")
print(f"Generated {len(vix_data)} days of VIX data")

# Display sample data
print("\nCDX Sample:")
print(cdx_data.head())

print("\nETF Sample:")
print(etf_data.head())

print("\nVIX Sample:")
print(vix_data.head())

2025-10-31 00:09:58,857 - macrocredit.data.sample_data - INFO - Generating CDX sample: index=CDX_IG, tenor=5Y, periods=252
2025-10-31 00:09:58,868 - macrocredit.data.sample_data - INFO - Generating VIX sample: periods=252
2025-10-31 00:09:58,871 - macrocredit.data.sample_data - INFO - Generating ETF sample: ticker=HYG, periods=252


Generated 252 days of CDX data
Generated 252 days of VIX data
Generated 252 days of ETF data

CDX Sample:
        date     spread      index tenor  series
0 2023-01-01  70.000000  CDX_IG_5Y    5Y      42
1 2023-01-02  70.914151  CDX_IG_5Y    5Y      42
2 2023-01-03  67.702784  CDX_IG_5Y    5Y      42
3 2023-01-04  70.183859  CDX_IG_5Y    5Y      42
4 2023-01-05  72.987167  CDX_IG_5Y    5Y      42

VIX Sample:
        date      close
0 2023-01-01  16.000000
1 2023-01-02  17.356357
2 2023-01-03  26.561353
3 2023-01-04  25.010464
4 2023-01-05  22.091705

ETF Sample:
        date      close ticker
0 2023-01-01  75.880203    HYG
1 2023-01-02  75.949926    HYG
2 2023-01-03  76.156336    HYG
3 2023-01-04  76.859569    HYG
4 2023-01-05  77.375789    HYG


## 3. Merge Market Data

Use standard pandas operations to combine data sources.

In [3]:
# Prepare data for merging
cdx_prep = cdx_data[["date", "spread"]].rename(columns={"spread": "cdx_spread"})
vix_prep = vix_data[["date", "close"]].rename(columns={"close": "vix_close"})
etf_prep = etf_data[["date", "close"]].rename(columns={"close": "etf_close"})

# Merge on date
market_data = cdx_prep.merge(vix_prep, on="date").merge(etf_prep, on="date")
market_data = market_data.set_index("date")

print(f"Merged market data: {len(market_data)} rows")
print(f"Columns: {list(market_data.columns)}")
print("\nMerged Data Sample:")
print(market_data.head())

Merged market data: 252 rows
Columns: ['cdx_spread', 'vix_close', 'etf_close']

Merged Data Sample:
            cdx_spread  vix_close  etf_close
date                                        
2023-01-01   70.000000  16.000000  75.880203
2023-01-02   70.914151  17.356357  75.949926
2023-01-03   67.702784  26.561353  76.156336
2023-01-04   70.183859  25.010464  76.859569
2023-01-05   72.987167  22.091705  77.375789


## 4. Compute cdx_etf_basis Signal

Calculate credit-equity basis signal that identifies relative value opportunities
between CDX index and credit ETF markets.

**Signal Convention**: Positive = Long credit risk (buy CDX/sell protection)

In [ ]:
# Configure signal parameters
signal_config = SignalConfig(
    lookback=20,
    min_periods=10,
)

# Prepare DataFrames for signal computation
cdx_df = market_data[["cdx_spread"]].rename(columns={"cdx_spread": "spread"})
etf_df = market_data[["etf_close"]].rename(columns={"etf_close": "close"})

# Compute CDX-ETF basis signal
signal = compute_cdx_etf_basis(
    cdx_df=cdx_df,
    etf_df=etf_df,
    config=signal_config,
)

# Add signal to market data
market_data["cdx_etf_basis"] = signal

print("Computed cdx_etf_basis signal")
print("\nSignal Statistics:")
print(market_data[["cdx_etf_basis"]].describe())

2025-10-31 00:09:58,902 - macrocredit.models.signals - INFO - Computing CDX-ETF basis: cdx_rows=252, etf_rows=252, lookback=20
2025-10-31 00:09:58,905 - macrocredit.models.signals - INFO - Computing CDX-VIX gap: cdx_rows=252, vix_rows=252, lookback=20
2025-10-31 00:09:58,907 - macrocredit.models.signals - INFO - Computing spread momentum: cdx_rows=252, lookback=20


Computed individual signals

Signal Statistics:
       cdx_etf_basis  cdx_vix_gap  spread_momentum
count     243.000000   234.000000       232.000000
mean       -0.101078    -0.013305         0.010373
std         1.167607     1.118674         1.932439
min        -2.772213    -2.886740        -5.711461
25%        -0.956350    -0.831705        -1.408021
50%        -0.078569     0.108475        -0.040037
75%         0.666807     0.779734         1.424015
max         3.328297     3.132649         5.096826


## 5. Visualize Signal

Plot cdx_etf_basis signal to understand strategy driver.

In [ ]:
# Plot signal
fig_signal = plot_signal(
    signal=market_data["cdx_etf_basis"],
    title="CDX-ETF Basis Signal",
    threshold_lines=[1.5, -1.5],
)
fig_signal.show()

print("Signal visualization complete")

2025-10-30 00:29:56,103 - macrocredit.visualization.plots - INFO - Plotting signal: 252 observations


Signal visualization complete


## 6. Run Backtest

Execute strategy backtest with position limits and transaction costs.

In [ ]:
# Configure backtest
backtest_config = BacktestConfig(
    entry_threshold=1.5,
    exit_threshold=0.75,
    position_size=10.0,  # $10M notional
    transaction_cost_bps=1.0,  # 1 bps
    max_holding_days=None,  # No time limit
    dv01_per_million=4750.0,
)

# Run backtest
backtest_result = run_backtest(
    composite_signal=market_data["cdx_etf_basis"],
    spread=market_data["cdx_spread"],
    config=backtest_config,
)

print("Backtest complete")
print(f"\nBacktest Results: {len(backtest_result.positions)} periods")
print("\nPosition Data Sample:")
print(backtest_result.positions.head())
print("\nP&L Data Sample:")
print(backtest_result.pnl.head())
print(f"\nFinal Cumulative P&L: ${backtest_result.pnl['cumulative_pnl'].iloc[-1]:,.2f}")

2025-10-30 00:30:02,549 - macrocredit.backtest.engine - INFO - Starting backtest: dates=252, entry_threshold=1.50, position_size=10.0MM
2025-10-30 00:30:02,559 - macrocredit.backtest.engine - INFO - Backtest complete: trades=2, total_pnl=$776436, avg_per_trade=$388218


Backtest complete

Backtest Results: 232 periods

Position Data Sample:
              signal  position  days_held     spread
date                                                
2023-01-21  0.542407         0          0  71.129851
2023-01-22  0.749442         0          0  70.462279
2023-01-23  0.093851         0          0  68.373262
2023-01-24  0.530566         0          0  72.203560
2023-01-25  0.931339         0          0  71.519616

P&L Data Sample:
            spread_pnl  cost  net_pnl  cumulative_pnl
date                                                 
2023-01-21         0.0   0.0      0.0             0.0
2023-01-22         0.0   0.0      0.0             0.0
2023-01-23         0.0   0.0      0.0             0.0
2023-01-24         0.0   0.0      0.0             0.0
2023-01-25         0.0   0.0      0.0             0.0

Final Cumulative P&L: $776,435.79


## 7. Calculate Performance Metrics

Compute Sharpe ratio, max drawdown, and other key statistics.

In [ ]:
# Calculate performance metrics
metrics = compute_performance_metrics(
    pnl_df=backtest_result.pnl,
    positions_df=backtest_result.positions,
)

print("Performance metrics calculated")
print("\n" + "="*50)
print("PERFORMANCE SUMMARY")
print("="*50)
print(f"Total Return:        ${metrics.total_return:>12,.2f}")
print(f"Sharpe Ratio:        {metrics.sharpe_ratio:>12.2f}")
print(f"Sortino Ratio:       {metrics.sortino_ratio:>12.2f}")
print(f"Max Drawdown:        ${metrics.max_drawdown:>12,.2f}")
print(f"Calmar Ratio:        {metrics.calmar_ratio:>12.2f}")
print(f"Hit Rate:            {metrics.hit_rate:>12.1%}")
print(f"Number of Trades:    {metrics.n_trades:>12d}")
print(f"Avg Holding Days:    {metrics.avg_holding_days:>12.1f}")
print(f"Avg Win:             ${metrics.avg_win:>12,.2f}")
print(f"Avg Loss:            ${metrics.avg_loss:>12,.2f}")
print(f"Win/Loss Ratio:      {metrics.win_loss_ratio:>12.2f}")
print(f"Annualized Return:   ${metrics.annualized_return:>12,.2f}")
print(f"Annual Volatility:   ${metrics.annualized_volatility:>12,.2f}")
print("="*50)

2025-10-30 00:30:10,383 - macrocredit.backtest.metrics - INFO - Computing performance metrics
2025-10-30 00:30:10,387 - macrocredit.backtest.metrics - INFO - Metrics computed: sharpe=1.89, max_dd=$-1000, hit_rate=100.0%


Performance metrics calculated

PERFORMANCE SUMMARY
Total Return:        $  776,435.79
Sharpe Ratio:                1.89
Sortino Ratio:               0.00
Max Drawdown:        $   -1,000.00
Calmar Ratio:              843.37
Hit Rate:                  100.0%
Number of Trades:               2
Avg Holding Days:             0.8
Avg Win:             $  227,901.47
Avg Loss:            $        0.00
Win/Loss Ratio:              0.00
Annualized Return:   $  843,369.91
Annual Volatility:   $  446,610.64


## 8. Visualize Results

Create interactive plots for equity curve and drawdown analysis.

In [ ]:
import plotly.express as px

# Plot positions over time (simple line chart)
fig_positions = px.line(
    backtest_result.positions,
    y="position",
    title="Position History",
    labels={"position": "Position (+1=Long, -1=Short, 0=Flat)", "date": "Date"},
)
fig_positions.show()

print("Position visualization complete")

Position visualization complete


In [ ]:
# Plot P&L evolution (equity curve)
fig_pnl = plot_equity_curve(
    pnl=backtest_result.pnl["net_pnl"],
    title="Cumulative P&L",
    show_drawdown_shading=True,
)
fig_pnl.show()

print("P&L visualization complete")

2025-10-30 00:30:20,665 - macrocredit.visualization.plots - INFO - Plotting equity curve: 232 observations


P&L visualization complete


In [ ]:
# Plot drawdown analysis
fig_drawdown = plot_drawdown(
    pnl=backtest_result.pnl["net_pnl"],
    title="Drawdown Analysis",
    show_underwater_chart=True,
)
fig_drawdown.show()

print("Drawdown visualization complete")

2025-10-30 00:30:30,884 - macrocredit.visualization.plots - INFO - Plotting drawdown: 232 observations


Drawdown visualization complete


## 9. Save Results

Save backtest results and metrics for further analysis.

In [ ]:
from macrocredit.persistence.parquet_io import save_parquet
from macrocredit.persistence.json_io import save_json
from dataclasses import asdict

# Create output directory
output_dir = Path("data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Save backtest results (positions and P&L)
positions_path = output_dir / "backtest_positions.parquet"
save_parquet(backtest_result.positions, positions_path)

pnl_path = output_dir / "backtest_pnl.parquet"
save_parquet(backtest_result.pnl, pnl_path)

# Save performance metrics
metrics_path = output_dir / "performance_metrics.json"
save_json(asdict(metrics), metrics_path)

print(f"Results saved to {output_dir}")
print(f"   - Positions: {positions_path}")
print(f"   - P&L: {pnl_path}")
print(f"   - Metrics: {metrics_path}")

2025-10-30 00:30:42,174 - macrocredit.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=data\processed\backtest_positions.parquet, rows=232, columns=4, compression=snappy
2025-10-30 00:30:42,189 - macrocredit.persistence.parquet_io - INFO - Saving DataFrame to Parquet: path=data\processed\backtest_pnl.parquet, rows=232, columns=4, compression=snappy
2025-10-30 00:30:42,194 - macrocredit.persistence.json_io - INFO - Saving JSON to data\processed\performance_metrics.json (13 top-level keys)


Results saved to data\processed
   - Positions: data\processed\backtest_positions.parquet
   - P&L: data\processed\backtest_pnl.parquet
   - Metrics: data\processed\performance_metrics.json


## Summary

This notebook demonstrated the complete end-to-end workflow:

1. Generated synthetic market data (CDX, ETF)
2. Computed cdx_etf_basis signal
3. Executed systematic backtest with risk controls
4. Calculated performance metrics
5. Visualized results with interactive plots
6. Exported results for persistence

### Next Steps

- Evaluate other signals independently (cdx_vix_gap, spread_momentum)
- Adjust position sizing and risk limits in `BacktestConfig`
- Test different CDX indices (HY, XO) and tenors
- Extend to multi-tenor strategies
- Compare signal performance metrics across ideas

### Key Takeaways

- All components are **modular** and **composable**
- Data generation is **deterministic** (reproducible with seeds)
- Signals follow **consistent sign convention** (positive = long credit)
- Visualization provides **actionable insights** into strategy behavior
- Framework supports **rapid iteration** on investment ideas
- Each signal evaluated independently for clear attribution